# What is the perfect anatomy of a powerlifter?

[Data source](https://openpowerlifting.gitlab.io/opl-csv/bulk-csv.html)

In [ ]:
import pandas as pd

df = pd.read_csv('data/openpowerlifting-2025-05-03-c77c7449.csv')

print(df.head())

print(df.info())

print(df.describe(include='all'))

print(df.isnull().sum())

In [ ]:
# Top lifters by total weight lifted
top_lifters = df[df['TotalKg'].notnull()].sort_values(by='TotalKg', ascending=False).head(10)
print(top_lifters[['Name', 'Sex', 'TotalKg', 'WeightClassKg', 'Federation']])

# Average lifts by gender
avg_by_sex = df.groupby('Sex')[['Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 'TotalKg']].mean()
print(avg_by_sex)

# Most common federations
print(df['Federation'].value_counts().head(10))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Distribution of TotalKg
sns.histplot(df['TotalKg'].dropna(), bins=50, kde=True)
plt.title("Distribution of TotalKg")
plt.xlabel("Total Weight Lifted (Kg)")
plt.show()

# Compare Dots scores by Equipment
sns.boxplot(x='Equipment', y='Dots', data=df[df['Dots'].notnull()])
plt.title("Dots Score by Equipment Type")
plt.xticks(rotation=45)
plt.show()

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

def get_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

image_path = 'image.jpg'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = pose.process(image_rgb)

if results.pose_landmarks:
    h, w, _ = image.shape
    landmarks = results.pose_landmarks.landmark

    def get_xy(index):
        lm = landmarks[index]
        return [lm.x * w, lm.y * h]

    schulter = get_xy(mp_pose.PoseLandmark.LEFT_SHOULDER.value)
    ellenbogen = get_xy(mp_pose.PoseLandmark.LEFT_ELBOW.value)
    handgelenk = get_xy(mp_pose.PoseLandmark.LEFT_WRIST.value)

    huefte = get_xy(mp_pose.PoseLandmark.LEFT_HIP.value)
    knie = get_xy(mp_pose.PoseLandmark.LEFT_KNEE.value)
    fuss = get_xy(mp_pose.PoseLandmark.LEFT_ANKLE.value)

    hals = get_xy(mp_pose.PoseLandmark.NOSE.value)

    oberarm = get_distance(schulter, ellenbogen)
    unterarm = get_distance(ellenbogen, handgelenk)
    oberarmlänge = oberarm + unterarm

    oberschenkel = get_distance(huefte, knie)
    unterschenkel = get_distance(knie, fuss)
    beinelänge = oberschenkel + unterschenkel

    oberkörperlänge = get_distance(hals, huefte)

    print(f"Armlänge / Beinlänge: {oberarmlänge / beinelänge:.2f}")
    print(f"Oberkörper / Beinlänge: {oberkörperlänge / beinelänge:.2f}")
else:
    print("Keine Person erkannt.")
